# Using the supporting files

This tutorial is accompanied by a set of supporting files. These exist in the directory titled
### Case Study 1 files
To use these files in this notebook:
1. Download the directory 'Case Study 1 files'
1. Download this notebook
1. Place the notebook inside the directory

Once this is done, you can run the notebook as intended.

In [ ]:
!pip3 install gwcloud-python bilby

# Running and comparing two Bilby jobs

In this tutorial, we will run two Bilby jobs, download their results and then analyse them.

The first step is to authenticate with the GWCloud service by using your API token.

In [ ]:
from gwcloud_python import GWCloud

gwc = GWCloud(token='Your token goes here') 

Once authenticated, we can submit Bilby jobs to a specified cluster. We have to provide a job name and description, along with whether or not the job should be private. We also supply standard Bilby ini files, which will specify everything needed to run the parameter estimation. These ini files and their supporting files can be found in the 'Case Study 1 files' directory. Please make sure to place all of the files within that directory into the same directory from which you run this code. Finally, we use the Cluster enum to submit the jobs to the Caltech cluster.

In [ ]:
from gwcloud_python import Cluster

spin_job = gwc.start_bilby_job_from_file(
    job_name="GW150914Example",
    job_description="GW150914 job with a spin prior",
    private=False,
    ini_file='GW150914_example.ini',
    cluster=Cluster.CIT
)

no_spin_job = gwc.start_bilby_job_from_file(
    job_name="GW150914NoSpin",
    job_description="GW150914 job with a no spin prior",
    private=False,
    ini_file='GW150914NoSpin.ini',
    cluster=Cluster.CIT
)

If you receive a job repetition error, simply change the job_name field. This error means you have already submitted a job with that job_name.

Please note, these jobs will take up to three days to run in full on the cluster.
We can check the status of the jobs, to determine whether they are running or have been completed.

In [ ]:
print(spin_job.status)
print(no_spin_job.status)

We now wish to set the event IDs of the above jobs. As both jobs are run on GW150914 data (with different priors), they will both have the GW150914_000000 event ID.

In [ ]:
spin_job.set_event_id(event_id='GW150914_000000')
no_spin_job.set_event_id(event_id='GW150914_000000')

If we lose reference to our jobs, we can obtain them again from the list of our jobs. In this instance, we're selecting the last two jobs we ran.

In [ ]:
jobs = []

user_jobs = gwc.get_user_jobs()
for job in user_jobs:
    if job.name == 'GW150914NoSpin' or job.name == 'GW150914Example':
        jobs.append(job)

Once the jobs have been completed, we can obtain the results files.

To obtain the log Bayes factor and create the superimposed plots, we must download the merged json file for each job (which has a size of roughly 1GB), and then read this file using bilby.core.result.read_in_result().

In [ ]:
from pathlib import Path

save_dir = Path('./files')

for job in jobs:
    job.save_result_json_files(save_dir)

Finally, we use Bilby to create the result objects from the merged json files. This allows us to make plots.

In [ ]:
import bilby, glob, math

for file in save_dir.glob('result/*.json'):
    if 'NoSpin' in file.name:
        no_spin_result = bilby.core.result.read_in_result(filename=file)
    else:
        spin_result = bilby.core.result.read_in_result(filename=file)

spin_ln_bf = spin_result.log_bayes_factor
nospin_ln_bf = no_spin_result.log_bayes_factor
BF = math.exp(nospin_ln_bf-spin_ln_bf)
print("The no-spin/spin Bayes factor is {0}".format(BF))

bilby.core.result.plot_multiple([spin_result, no_spin_result], labels = ['Spin', 'No Spin'], parameters = ['mass_1', 'mass_2', 'mass_ratio', 'theta_jn', 'luminosity_distance'])
